In [30]:
from pesq import pesq
import librosa
import numpy as np

# SIR (Signal to Interference Ratio)


```MATLAB
% Copyright 2008 Emmanuel Vincent
% This software is distributed under the terms of the GNU Public License
% version 3 (http://www.gnu.org/licenses/gpl.txt)
% If you find it useful, please cite the following reference:
% Emmanuel Vincent, C�dric F�votte and R�mi Gribonval, "Performance
% measurement in blind audio source separation," IEEE Trans. on Audio,
% Speech and Language Processing, 14(4):1462-1469, 2006.
```

https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=1643671

In [32]:
estim=[]

target=[]
for i in range(4) : 
    tmp_target, _ = librosa.load("target_"+str(i)+".wav",sr=16000)
    tmp_estim, _  = librosa.load("39_"+str(i)+".wav",sr=16000)

    target.append(tmp_target)
    estim.append(tmp_estim)

s_target = []
e_interf = []
    
for i in range(4) : 
    s_target.append(np.inner(estim[i],target[i])*target[i]/np.inner(target[i],target[i]))
    
    tmp = None
    for j in range(4) : 
        if i == j :
            continue
        if tmp is None : 
            tmp = np.inner(estim[i],target[j])*target[j]/np.inner(target[j],target[j])
        else : 
            tmp += np.inner(estim[i],target[j])*target[j]/np.inner(target[j],target[j])
    e_interf.append(tmp)


for i in range(4) : 
    print(i)
    print(s_target[i].shape)
    print(e_interf[i].shape)
    SIR = 10*np.log10((np.inner(s_target[i],s_target[i]))/np.inner(e_interf[i],e_interf[i]))
    print("SIR : {}".format(SIR))

    print("PESQ(wb) : {}".format(pesq(16000, target[i], estim[i], 'wb')))
    print("PESQ(nb) : {}".format(pesq(16000, target[i], estim[i], 'nb')))
    

0
(16000,)
(16000,)
SIR : 19.378488063812256
PESQ(wb) : 1.2397656440734863
PESQ(nb) : 1.6209020614624023
1
(16000,)
(16000,)
SIR : 18.61210823059082
PESQ(wb) : 2.105191469192505
PESQ(nb) : 2.6784563064575195
2
(16000,)
(16000,)
SIR : 9.921001195907593
PESQ(wb) : 1.8335928916931152
PESQ(nb) : 2.3431551456451416
3
(16000,)
(16000,)
SIR : 15.172652006149292
PESQ(wb) : 1.7513091564178467
PESQ(nb) : 1.973534345626831


bss_eval_toolkit이랑 SIR은 다른데 MATLAB 코드에서는 window 별로 projection 해서 그렇게 되는것 같다. 
다만 경향성은 동일하기 때문에 괜찮지 않을까? 

## SIR & PESQ for Batch Multi-target Multi-channel  

clean = [B,N,C,T]  
estim = [B,N,C,T]  

In [ ]:
"""
    clean : [B,N,C,T]
    estim : [B,N,C,T]
"""
def SIR(clean,estim) : 
    SIR = 0
    # for Batch
    for b in range(clean.shape[0]) : 
        # for Target
        for n in range(clean.shape[1]) : 
            
            s_target = []
            e_interf = []
            
            # for Channel - s
            for c in range(clean.shape[2]) : 
                s_target.append(np.inner(clean[b,n,c,:],target[b,n,c,:])*target[b,n,c,:]/np.inner(target[b,n,c,:],target[b,n,c,:]))
                
                # for Channel  - i
                tmp = None
                for j in range(clean.shae[2]) : 
                    if c == j :
                        continue
                    if tmp is None : 
                        tmp = np.inner([b,n,c,:],target[b,n,j,:])*target[b,n,j,:]/np.inner(target[b,n,j,:],target[b,n,j,:])
                    else : 
                        tmp += np.inner([b,n,c,:],target[b,n,j,:])*target[b,n,j,:]/np.inner(target[b,n,j,:],target[b,n,j,:])
                e_interf.append(tmp)
            
            # for Channel - SIR
            for c in range(clean.shape[2]) :     
                SIR += 10*np.log10((np.inner(s_target[c],s_target[c]))/np.inner(e_interf[c],e_interf[c]))
    # mean
    SIR /= clean.shape[0]*clean.shape[1]*clean.shape[2]
    
    
"""
    clean : [B,N,C,T]
    estim : [B,N,C,T]
"""    
def PESQ(clean,estim) : 
    def SIR(clean,estim) : 
    SIR = 0
    # for Batch
    for b in range(clean.shape[0]) : 
        # for Target
        for n in range(clean.shape[1]) : 
            
            s_target = []
            e_interf = []
            
            # for Channel - s
            for c in range(clean.shape[2]) : 